In [ ]:
import cv2 as cv
import numpy as np
import epfl_data2 as epfl_data
import matplotlib.pyplot as plt
from skimage.feature import hog as HoG
from sklearn.preprocessing import normalize
from sklearn.utils import shuffle

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [ ]:
2299-972

In [ ]:
def flip_angle(angle):
    if angle == 180:
        return 0
    elif angle == 0:
        return 180
    else:
        return 360-angle


rotations = 3
bins = 6

data = epfl_data.Data(rotations,bins)


training_images = np.zeros((1327,11016))
training_images_f = np.zeros((1327,11016))
test_images = np.zeros((972,11016))
training_tgts_1 = []
training_tgts_2 = []
training_tgts_3 = []
training_tgts_f1 = []
training_tgts_f2 = []
training_tgts_f3 = []
train_angles = []
train_angles_f = []
test_tgts = []

height = 250
width = 376

for d in range(len(data.samples[0])*2):
    image = cv.imread(data.samples[0][d])
    image = image[28:212, 30:348]
    image = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    f_image = np.fliplr(image)
    if d <= 1326:
        image = HoG(image, orientations=8, pixels_per_cell=(16,16))
        image = normalize(image)
        f_image = HoG(f_image, orientations=8, pixels_per_cell=(16,16))
        f_image = normalize(image)
        training_images[d] = image
        training_tgts_1.append(data.samples[2][d][0])
        training_tgts_2.append(data.samples[2][d][1])
        training_tgts_3.append(data.samples[2][d][2])
        training_images_f[d] = f_image
        training_tgts_f1.append(data.samples[2][d][0])
        training_tgts_f2.append(data.samples[2][d][1])
        training_tgts_f3.append(data.samples[2][d][2])
        
        train_angles.append(np.deg2rad(data.samples[1][d]))
        train_angles_f.append(np.deg2rad(flip_angle(data.samples[1][d])))
    else:
        if d-1326 == 972:
            break
        else:
            image = HoG(image, orientations=8, pixels_per_cell=(16, 16))
            image = normalize(image)
            test_images[d-1326] = image
            test_tgts.append(np.deg2rad(data.samples[1][d]))

train = np.append(training_images, training_images_f,axis=0)
for i in range(len(training_tgts_1)):
    training_tgts_1.append(training_tgts_f1[i])
    training_tgts_2.append(training_tgts_f2[i])
    training_tgts_3.append(training_tgts_f3[i])
for i in range(len(train_angles)):
    train_angles.append(train_angles_f[i])
print('done')

In [ ]:
print(data.samples[3][:])

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Input
from keras.layers.convolutional import Conv1D
from keras.models import Model
from keras.losses import categorical_crossentropy as cross

In [ ]:
hog_input = Input(shape=(11016,), name='input')
layers = Dense(1000, input_dim=11016, activation='relu')(hog_input)
layers = Dropout(.5)(layers)
layers = Dense(100, activation='relu')(layers)
layers = Dropout(.5)(layers)
layers = Dense(10, activation='relu')(layers)
layers = Dropout(.5)(layers)
outputs = []
for r in range(rotations):
    outputs.append(Dense(bins, activation='softmax', name='output{}'.format(r))(layers))

In [ ]:
model = Model(inputs=hog_input, outputs=outputs)
model.compile(optimizer='adam', loss=cross, metrics=['accuracy'])
model.fit(train, 
          [np.asarray(training_tgts_1), np.asarray(training_tgts_2), np.asarray(training_tgts_3)], 
          batch_size=32, epochs=20, verbose=1)

In [ ]:
second_round = model.predict(test_images[:-1])

In [ ]:
second_array = np.asarray(second_round)
second_array = np.moveaxis(second_array, 1,0)
flat_second = second_array.reshape((-1, 18))

In [ ]:
vonMises = Sequential()
vonMises.add(Dense(18, input_dim=18, activation='relu'))
vonMises.add(Dropout(.5))
vonMises.add(Dense(15, activation='relu'))
vonMises.add(Dropout(.5))
vonMises.add(Dense(10, activation='relu'))
vonMises.add(Dropout(.5))
vonMises.add(Dense(5, activation='relu'))
vonMises.add(Dropout(.5))
vonMises.add(Dense(5, activation='relu'))
vonMises.add(Dropout(.5))
vonMises.add(Dense(1))

In [ ]:
vonMises.compile(optimizer='adam', loss='mse', metrics=['accuracy'])
vonMises.fit(flat_second, 
             np.asarray(test_tgts),
             batch_size=32, epochs=50, verbose=1)

In [ ]:
outputTest = model.predict(test_images[:-1])
outputTest = np.asarray(outputTest)
outputTest = np.moveaxis(outputTest, 1,0)
outputTest = outputTest.reshape((-1,18))
print(len(outputTest))
print(len(test_tgts))
score2 = vonMises.predict(outputTest)



print("PREDICTIONS 1-10:", score2[:10].T)
print('\n')
print("ACTUAL 1-10:", test_tgts[:10])





In [ ]:
import math
from matplotlib import pyplot as plt
from vonmiseskde import VonMisesKDE

discrete_orientations = data.samples[3][1328:]

results = np.zeros((len(discrete_orientations),2))

# Kernel density estimator
kappa = 10
print(len(discrete_orientations), len(flat_second), len(test_images[:-1]))
for i in range(len(flat_second)):
    kde = VonMisesKDE(np.deg2rad(discrete_orientations[i]), weights=np.asarray(flat_second[i]), kappa=kappa)

    # Input for test points
    test_x = np.linspace(0, 2*math.pi, 720)

    # Display posterior estimate
    #plt.plot(test_x, kde.evaluate(test_x), zorder=20)
    #plt.title('von Mises density for a sample image')
    maxi = np.argmax(kde.evaluate(test_x))
    #print("Radians: {}, Degree: {}, True Deg: {}".format(test_x[maxi], np.rad2deg(test_x[maxi]), data.samples[1873+i][1]))
    results[i,0] = np.rad2deg(test_x[maxi])
    results[i,1] = data.samples[1][1327+i]
    #plt.xlabel("Orientation")
    #plt.xlim(0, 2*math.pi)
    #plt.ylim(0, 1)
    #plt.show()

def circular_loss(y, y_pred):
    loss = 180 - np.absolute(np.absolute(y_pred - y) - 180)
    return loss

print(results)

error = circular_loss(results[:,1], results[:,0])

plt.hist(error, 30)
plt.title("Distribution of Error")
plt.xlabel("Error Magnitude")
plt.ylabel("Frequency")
plt.show()

print("Mean error: {}".format(np.mean(error)))
print("Median error: {}".format(np.median(error)))
